In [1]:
import os
import h5py
import pickle
import numpy as np
import napari
from datetime import date
from skimage import morphology
from skimage.filters import gaussian
from skimage.exposure import rescale_intensity
from skimage.transform import resize
from pystackreg import StackReg
from scipy import ndimage as ndi
from scipy.stats import zscore

In [2]:
def matrix_rc2xy(affine_matrix):
    swapped_cols = affine_matrix[:, [1, 0, 2]]
    swapped_rows = swapped_cols[[1, 0, 2], :]
    return swapped_rows


def find_translation(source,destination):
    
    sr = StackReg(StackReg.TRANSLATION)
    tmat = sr.register(destination,source)
    
    tmat = matrix_rc2xy(tmat)
    
    return tmat

def find_scaled_rotation(source,destination):
    
    sr = StackReg(StackReg.SCALED_ROTATION)
    tmat = sr.register(destination,source)
    
    tmat = matrix_rc2xy(tmat)

    return tmat

In [3]:
# check date

today = date.today()
today = today.strftime("%Y%m%d")
today

'20220815'

In [4]:
im_path = r'W:\Users\kmkedz\Dragonfly PSFs\Dual Cam\DualCamPSF G-FR 63X 1-5Xmag_2022-04-25_12.02.32_single_frame.ims' 

save_dir = r'W:\Users\kmkedz\Dragonfly PSFs'
save_file = f'{today}_2D_zyla_5x_63x_fr_g' #f'{today}_MSL_zyla_20x'
save_file

'20220815_2D_zyla_5x_63x_fr_g'

In [5]:
channel_names = ['ch_488','ch_637']
vis_colors = ['green','magenta']#['magenta','red','green','blue'] #

ch_num = len(channel_names)

## Read handles to stacks

In [6]:
f = h5py.File(im_path,'r')

In [7]:
for i in range(ch_num):

    stack = f['DataSet']['ResolutionLevel 0']['TimePoint 0'][f'Channel {i}']['Data']
    exec("stack_"+channel_names[i]+"= stack")
    
if (len(stack.shape) == 2) or (stack.shape[0] == 1):
    
    print(f'Dimensions of the selected stack: {stack.shape}')

else:
    
    print(f'Warning - only 2D transformation will be calculated!!!!')

Dimensions of the selected stack: (1, 1024, 1024)


## Alignment in xy

In [8]:
# make projection in xy

pos = 50
rad = 30

for i in range(ch_num):
    
    exec("stack = stack_"+channel_names[i])
    
    if (len(stack.shape) == 2) or (stack.shape[0] == 1):
        
        xy = np.squeeze(stack)
        
    else:
    
        xy = np.max(stack[pos-rad:pos+rad,:,:],axis=0)
    
    exec("xy_"+channel_names[i]+"= xy")

In [9]:
# processing of xy images

sigma = 2
footprint = morphology.disk(3)

xy_list = []

for i in range(ch_num):
    
    exec("xy = xy_"+channel_names[i])
    
    xy = morphology.white_tophat(xy, footprint)
    xy = gaussian(xy, sigma)
    
    # normalize
    xy = zscore(xy,axis=None)
    
    xy_list.append(xy)
    
    exec("xy_"+channel_names[i]+"= xy")

In [10]:
# visualize original xy data

viewer = napari.Viewer()

for i in range(ch_num):
    
    exec("im = xy_"+channel_names[i])
    viewer.add_image(im,blending='additive',colormap=vis_colors[i],name = channel_names[i])

In [11]:
# alignment in xy plane

destination = xy_list[0]

tmat_xy_list = []

for i in range(ch_num):
    
    exec("source = xy_"+channel_names[i])
    
    tmat = find_scaled_rotation(source,destination)
    
    ch_aligned = ndi.affine_transform(source, tmat)
    
    exec("xy_"+channel_names[i]+"_aligned = ch_aligned")
    
    tmat_xy_list.append(tmat)


In [12]:
# check the alignment in xy

s_fact = 99.99

viewer = napari.Viewer()

for i in range(ch_num):
    
    exec("im = xy_"+channel_names[i]+"_aligned")
    viewer.add_image(im,blending='additive',colormap=vis_colors[i],contrast_limits=(0,np.percentile(im,s_fact)),
                    name = channel_names[i])

## Save transformations

In [13]:
with open(os.path.join(save_dir,save_file+'.pkl'),"wb") as f:
    pickle.dump([channel_names,tmat_xy_list],f)


with open(os.path.join(save_dir,save_file+'.txt'), "w") as f:
    
    f.write(f'Alignment calculated {today}.')
    f.write('\n')
    f.write('\n')
    
    f.write(f'Channels:')
    f.write('\n')
    f.write('\n')

    for tmat in channel_names:
        f.write(tmat)
        f.write('\n')

        
    f.write('\n')
    f.write(f'Alignment in xy:')
    f.write('\n')
    f.write('\n')

    for tmat in tmat_xy_list:
        np.savetxt(f, tmat, delimiter=',')
        f.write('\n')